<h1><center> UFU - Federal University of Uberlândia</center></h1>

<h2><center>Undergraduate Program in Civil Engineering</center></h2>

<h3><center>SCIENTIFIC RESEARCH PROJECT</center><br>
TITLE: USING XGBOOST MODELS FOR DAILY RAINFALL PREDICTION 
<br>  
<br>  
STUDENT: Pedro Augusto Toledo Rios</h3>

<p>This notebook is part of a Scientific Research Project in the field of Computer Science/Data Analysis.</p>


#  Notebook for Determining the Daily Rainfall Amount (mm) - Dry Period

## Imports and Initial Configurations

In [ ]:
# Data Analysis and Wrangling
import pandas as pd
import numpy as np
import random as rnd

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

# Machine Learning - Scikit-Learn
from sklearn import metrics
from sklearn.linear_model import (
    LogisticRegression,  # Logistic regression model
    LinearRegression,    # Linear regression model
    Perceptron,          # Perceptron classifier (basic neural network)
    SGDClassifier        # Stochastic Gradient Descent classifier
)
from sklearn.svm import SVC, LinearSVC  # Support Vector Machine classifiers
from sklearn.ensemble import (
    RandomForestClassifier,  # Random Forest Classifier (ensemble method)
    AdaBoostClassifier       # Adaptive Boosting classifier
)
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors classifier
from sklearn.naive_bayes import GaussianNB  # Gaussian Naïve Bayes classifier
from sklearn.tree import DecisionTreeClassifier  # Decision Tree classifier
from sklearn.model_selection import train_test_split  # Splitting dataset into training and test sets
from sklearn.metrics import (
    precision_score,       # Precision metric
    recall_score,          # Recall metric
    f1_score,              # F1-score metric
    accuracy_score,        # Accuracy metric
    classification_report, # Summary report for classification results
    confusion_matrix,      # Confusion matrix
    ConfusionMatrixDisplay,# Confusion matrix visualization
    mean_absolute_error,   # Mean Absolute Error (MAE)
    mean_squared_error,    # Mean Squared Error (MSE)
    r2_score              # R-squared metric (coefficient of determination)
)

# Deep Learning - TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    LSTM,         # Long Short-Term Memory (LSTM) layer for sequential data
    Dense,        # Fully connected (dense) layer
    Dropout,      # Dropout layer to reduce overfitting
    Bidirectional,# Bidirectional LSTM layer
    SimpleRNN,    # Simple Recurrent Neural Network layer
    Embedding,    # Embedding layer for handling categorical data
    Masking       # Masking layer to ignore certain time steps in sequential models
)

# Pandas Configuration Settings
pd.set_option("display.max_colwidth", 150)  # Adjust maximum column width for display
pd.set_option("display.min_rows", 20)       # Adjust minimum number of rows to display


## Exploratory Data Analysis

In [ ]:
# Data Preprocessing

# Defining missing values representation in the dataset
missing_values = ['n/a', 'na', '*****', '*', '*******', ' -', '******', 
                  '5..84', '3..66', '3.3.21', '1..41', '********', '3.7.94', 
                  '354.59*', '564..79', '5.04.24', '21:36', '**********', 
                  '***', '*********', '03:18', '00:00', '03:48', '08:42', 
                  '03:06', '09:06', '01:30', '07:48', '09:12', '10:18', 
                  '01:24', '#VALUE!', '926,4923,8', '27/07/1902**21:36:00', 
                  '-', '926.4923.8', '185.488.992']

# Loading the dataset while treating missing values
dados_clima = pd.read_csv('C:/Users/auped/Desktop/IC CORREÇÕES/python 05-11/pesquisa/dadosclima_PERIODOSECO.csv', 
                          header=None, sep=';', na_values=missing_values)

# Renaming columns for better readability
dados_clima.columns = ['Max Temp (°C)', 'Min Temp (°C)', 'Avg Temp (°C)', 
                       'Wind Speed (km/h)', 'Solar Radiation (cal/cm²/h)', 
                       'Pressure (mb)', 'Humidity (%)', 'Rainfall (mm)', 
                       'Month', 'Year', 'Rainy/Dry']

# Replacing incorrect string patterns in numerical columns
dados_clima['Pressure (mb)'] = dados_clima['Pressure (mb)'].str.replace(',,', '.')
dados_clima['Year'] = dados_clima['Year'].str.replace(',,', '')

# Converting data types
dados_clima['Humidity (%)'] = dados_clima['Humidity (%)'].astype(float)
dados_clima['Pressure (mb)'] = dados_clima['Pressure (mb)'].astype(float)
dados_clima['Solar Radiation (cal/cm²/h)'] = dados_clima['Solar Radiation (cal/cm²/h)'].astype(float)
dados_clima['Year'] = dados_clima['Year'].astype(int)

# Dropping missing values in Humidity column
dados_clima['Humidity (%)'].dropna(inplace=True)

# Filtering out pressure values that fall outside the valid range
dados_clima = dados_clima[(dados_clima['Pressure (mb)'] >= 870) & 
                          (dados_clima['Pressure (mb)'] <= 1100)]

# Assigning the target variable
combine = [dados_clima]
rainfall = dados_clima['Rainfall (mm)']

# Displaying the first five rows of the preprocessed dataset
dados_clima.head()

# Dropping any remaining missing values
dados_clima.dropna(inplace=True)

# Displaying the number of missing values per column
print(dados_clima.isnull().sum())


### Implementation of a Rain Detection Class for Daily Weather Data

In [ ]:
# Loop to populate the new column with 0 (no rain) and 1 (rain)
for dados_clima in combine:    
    dados_clima.loc[dados_clima['Rainfall (mm)'] == 0, 'Rained?'] = 0
    dados_clima.loc[dados_clima['Rainfall (mm)'] > 0, 'Rained?'] = 1

# Displaying the first five rows of the dataset
dados_clima.head()


In [ ]:
# Statistical summary of the dataset
dados_clima.describe()


In [ ]:
# Rename columns to English
dados_clima_ingles = dados_clima.describe().rename(columns={
    'Temp Máx (°C)': 'Max Temperature (°C)',
    'Temp Mín (°C)': 'Min Temperature (°C)',
    'Temp Média (°C)': 'Average Temperature (°C)',
    'Velocidade do Vento (km/h)': 'Wind Speed (km/h)',
    'Radiação solar (cal/cm²/h)': 'Solar Radiation (cal/cm²/h)',
    'Pressão (mb)': 'Pressure (mb)',
    'UR (%)': 'Relative Humidity (%)',
    'Chuva (mm)': 'Rainfall (mm)',
    'Mês': 'Month',
    'Ano': 'Year',
    'Chuvoso/Seco': 'Rainy/Dry'
})

# Save the dataset summary to an Excel file
file_path = r"C:\Users\auped\Desktop\IC CORREÇÕES\climate_data_dry_period_summary.xlsx"
dados_clima_ingles.to_excel(file_path)

print(f"File successfully saved at: {file_path}")


In [ ]:
# Generate statistical summary of the dataset
df_summary = dados_clima.describe()

# Save the statistical summary to an Excel file
file_path = "statistical_summary_dry_period.xlsx"
df_summary.to_excel(file_path)

print(f"File successfully saved as: {file_path}")


In [ ]:
# Display the number of missing values in each column
print("\nMissing Values in the DataFrame:\n", dados_clima.isnull().sum(), sep="")


In [ ]:
# Handling Missing Values  

dados_clima['Min Temp (°C)'].fillna(method='ffill', inplace=True)  # Forward-fill to propagate the last observed value  
dados_clima['Avg Temp (°C)'].fillna(method='ffill', inplace=True)  
dados_clima['Wind Speed (km/h)'].fillna(method='ffill', inplace=True)  
dados_clima['Solar Radiation (cal/cm²/h)'].fillna(method='ffill', inplace=True)  
dados_clima['Humidity (%)'].fillna(method='ffill', inplace=True)  
dados_clima['Rainfall (mm)'].fillna(method='ffill', inplace=True)  
dados_clima['Pressure (mb)'].fillna(method='ffill', inplace=True)  
dados_clima['Rained?'].fillna(method='ffill', inplace=True)  
dados_clima['Max Temp (°C)'].fillna(method='ffill', inplace=True)  

# Verifying missing values after imputation  
print("\nMissing Values in the DataFrame after treatment:\n", dados_clima.isnull().sum(), sep="")


In [ ]:
# Compute Pearson correlation and sort values  
columns_corr = dados_clima.corr(method='pearson')['Rainfall (mm)'].sort_values()  
print(columns_corr)


In [ ]:
# Count rainy and dry days  
rain_count = dados_clima['Rained?'].value_counts()

# Results  
rainy_days = rain_count.get(1, 0)  # Returns 0 if no value 1 is found  
dry_days = rain_count.get(0, 0)  # Returns 0 if no value 0 is found  

print(f"Total rainy days: {rainy_days}")  
print(f"Total dry days: {dry_days}")


## Time Period Selection for Analysis

In [ ]:
start_year = 1980
end_year = 2020
train_start_year = 1983
train_end_year = 2008
test_start_year = 2009
test_end_year = 2019


In [ ]:
dados_clima.drop(dados_clima.loc[dados_clima['Rainfall (mm)']==0].index, inplace=True)


In [ ]:
dadosparateste = dados_clima.copy()

#Criando uma nova coluna de chuva para ficar no final do dataframe

qtdchuva = dadosparateste['Rainfall (mm)']


dadosparateste['Chuva att (mm)'] = qtdchuva

#Visualização das primeiras cinco linhas da tabela
dadosparateste.head()  

In [ ]:
dadosparateste.drop(columns = ['Chuva att (mm)'], axis = 1, inplace=True)


## Shared Utility Functions for All Models

In [ ]:
# Library para statmodels
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error

#Definir funções que serão utilizadas por todos os modelos de predição

#Imprimir mensagens de erro da predição
def imprimir_scores_predicao(strModel, y_target, arPredict):
  #print(strModel, ' - R2: ', r2_score(y_target, arPredict))
  print(strModel, ' - MAE:  ', mean_absolute_error(y_target, arPredict))
  print(strModel, ' - MAPE: ', mean_absolute_percentage_error(y_target, arPredict))
  #print(strModel, ' - MSE: ', mean_squared_error(y_target, arPredict))
  print(strModel, ' - RMSE: ', np.sqrt(mean_squared_error(y_target, arPredict)))
  print(strModel, ' - MSE: ',mean_squared_error(y_target, arPredict))
  print(strModel, ' - R2: ',r2_score(y_target, arPredict))
  #mse = metrics.mean_squared_error(y, yhat)
  
  
def imprimir_graficos_predicao(strModel, y_target, arPredict):
  
  df_Test_Predicted = y_target.copy()
  df_Test_Predicted['Daily Rainfall (mm)'] = arPredict

  #Plotagem dos dados de teste e dados previstos
  plt.figure(figsize=(16,8))
  plt.plot(y_target.loc['Dia'], y_target, color = 'blue', label = 'Chuva Real')
  plt.plot(y_target.loc['Dia'], arPredict, color = 'red', label = 'Chuva Prevista')
  font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 13.5,
        }
  plt.title('Comparação entre o real e o previsto pelo modelo', fontdict=font)
  plt.xlabel('Período')
  plt.ylabel('Quantidade de chuva')
  plt.legend()
  plt.grid(True)
  #plt.savefig('lstm2.pdf')
  plt.show() 

# Machine Learning Models - Regression

In [ ]:
# Create training and testing dataframes
df_train = dadosparateste[(dadosparateste['Year'] >= train_start_year) & 
                          (dadosparateste['Year'] <= train_end_year)]

df_test = dadosparateste[(dadosparateste['Year'] >= test_start_year) & 
                         (dadosparateste['Year'] <= test_end_year)]

# Split features (X) and target variable (y)
X_train = df_train.iloc[:, 0:7]
y_train = df_train[['Rainfall (mm)']]

X_test = df_test.iloc[:, 0:7]
y_test = df_test[['Rainfall (mm)']]


# XGBOOST 

In [ ]:
import xgboost as xgb

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 100, alpha = 1, n_estimators = 100000)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [ ]:
imprimir_scores_predicao("XGBoost", y_test, preds)


In [ ]:
y_test.describe()

In [ ]:
chuvaprevista = []
for z in range(len(preds)):
    #print(y_pred4[z])
    chuvaprevista.append(preds[z])
    
df_test['Previsão']= chuvaprevista
df_test.head()    

In [ ]:
df_test = df_test[df_test['Year']>= 1980]
df_test =  df_test[df_test['Year'] <= 2019]

In [ ]:
plt.figure(figsize=(10,10))

true_value = df_test['Rainfall (mm)']
predicted_value = df_test['Previsão']
plt.scatter(true_value, predicted_value, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predicted_value), max(true_value))
p2 = min(min(predicted_value), min(true_value))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()